In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from typing import Iterable, Tuple, List, Callable

In [ ]:
def objective_func(x):
    return x * np.sin(10 * np.pi * x) + 2.0

In [ ]:
CHROMOSOME_SIZE = 12
POPULATION_SIZE = 6
CROSSOVER_RATE = 0.8

x_min = -1
x_max = 2

x_range = np.linspace(x_min,x_max, 100)
y_range = objective_func(x_range)
plt.plot(x_range, objective_func(x_range))

In [ ]:
def initialize_genotype_population(population_size, chromo_size):
    
    x = np.random.randint(low = 0, high = 2, size = (population_size, chromo_size), dtype=bool)
    
    return x

In [ ]:
def binary_to_float(bin_arr, a = 0, b = 1):
    
    bit_num = bin_arr.shape[-1]
    
    dec = bin_arr @ np.power(2, np.arange(bit_num))
    
    zero_one_range = dec / (2**bit_num)
    
    real_arr = zero_one_range * (b - a) + a
    
    return real_arr

In [ ]:
def evaluate(x_phenotype: Iterable, objective_func: Callable):
    return objective_func(x_phenotype)

In [ ]:
def calc_relative_fittness(evals: Iterable):
    return evals/np.sum(evals)

In [ ]:
def create_mating_pool(genotype_arr, prob_arr):    
    indeces = np.random.choice(np.arange(len(prob_arr)), size=len(genotype_arr), replace=True, p=prob_arr)
    return genotype_arr[indeces]

In [ ]:
def single_point_crossover(x: List):
    
    x0 = x[0]
    x1 = x[1]
    
    point = np.random.randint(low = 1, high = x.shape[1])
    
    x1[:point], x0[:point] = x0[:point], x1[:point].copy()
    
    return x

In [ ]:
def flip_bit_mutate(x: np.array, mutation_rate: float = 0.1):
    
    probs = np.random.rand(x.shape[0], x.shape[1])
    
    mutat_mask = probs < mutation_rate
    
    x_mutated = x.chopy()
    
    x_mutated[mutat_mask] = ~x_mutated[mutat_mask]
    
    return x_mutated
    

In [ ]:
def split_to_chunks(x: Iterable, n: int = 2):
    '''
    splits x array into evenly sized chunks. 
    
    x: array to split into chunks.
    n: number of elements in chunks.
    '''
    assert len(x) % n == 0, 'length of x should be divisible on n with remainder equal to 0.'
    
    for i in range(0, len(x), n):
        yield x[i:i + n]  

In [ ]:
def create_next_generation(x: Iterable, splitter: Callable, crossover: Callable, mutate: Callable, crossover_rate: float = 0.5, mutation_rate: float = 0.4):
    '''
    Performs crossover of individuals.
    
    x: mating pool of individuals to perform crossover on (2d array). dim0 - is individuals, dim1 - genes;
    crossover_rate: number bewteen 0 and 1. probability that given two individuals perform a crossover. 
    if two individuals do not perform a crossover according to probability crossover_rate, their off-spring are themselves. 
    
    '''
    new_generation = []
    
    np.random.shuffle(x)
    
    chunks = splitter(x)

    for ch in chunks:
        
        if np.random.rand() < crossover_rate:
            ch = crossover(ch)
            
        ch = mutate(ch, mutation_rate)
            
        new_generation.extend(ch)
            
    return np.array(new_generation)

In [ ]:
x1 = np.random.randint(low=0, high=2,size=(1, 3, 12), dtype=bool)
x2 = np.array([[1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
               [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
               [1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1]], dtype=bool)
x2 =np.expand_dims(x2,axis=0)
x = np.concatenate((x1, x2), axis=0)
x.shape

In [ ]:
#create first binary population
x_genotype = initialize_genotype_population(10000, CHROMOSOME_SIZE)
x_genotype

In [ ]:
%%time
#convert binary population to float one

x_phenotype = binary_to_float(x_genotype, x_min, x_max)

In [ ]:
%%time
#evaluate each individual (calculate fittness values) according to objective function 

evals = evaluate(x_phenotype, objective_func)
evals.shape

In [ ]:
%%time
#using fittness values calculate probabilities of being selected for mating pool

rel_fittness_vals = calc_relative_fittness(evals)
rel_fittness_vals.shape

In [ ]:
mating_pool = create_mating_pool(x_genotype, rel_fittness_vals)

In [ ]:
mating_pool.shape

In [ ]:
current_gen = create_next_generation(mating_pool, split_to_chunks, single_point_crossover, flip_bit_mutate, crossover_rate=0.8, mutation_rate=0.6)

In [ ]:
current_gen.astype(int)

In [ ]:
mating_pool.astype(int)